# Setting up for PEST(++) analyses

In this notebook, we will use the MF6 model that was built using `modflow-setup`.  We will constuct the PEST interface (e.g. template files, instruction files, control file), as well as generate the prior parameter ensemble.  The best part is, this will all be done programmatically! That means whenever "issues" are discovered, it is easier to recover...

In [ ]:
import sys
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
sys.path.append('../../dependencies/')
import pandas as pd
import numpy as np
import matplotlib as mpl
font = {'size'   : 12}
mpl.rc('font', **font)
import flopy as fp
import pyemu




## preparing for `PstFrom`

The `PstFrom` class in `pyEMU` can automate the PEST(++) setup process.  `PstFrom` expects model input and output files to be either array format (2-d homogenous data type) or list format (heterogenous data type by columns).  So this means we need to get the MF6 model to use "external" format so that all input quantities we are interested in are stored in stand-alone files. Luckily, `flopy` can do this operation for us...

In [ ]:
# the original directory that holds the MF6 files
org_d = os.path.join("..","..","models","sgn_mfsetup_transport")
assert os.path.exists(org_d),"need to run mfsetup notebooks"
# the model base name
mname = "sgn_50"
assert os.path.exists(os.path.join(org_d,mname+".nam"))
# a temporary directory that will hold the model files
tmp_d = os.path.join("..","..","models","pstfrom_temp")
if os.path.exists(tmp_d):
    shutil.rmtree(tmp_d)

Load the existing model and convert it to "external" input files (arrays and lists).  Having the array and list inputs as external files makes everything in the pest world easier...

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws=org_d)
sim.simulation_data.mfpath.set_sim_path(tmp_d)
m = sim.get_model(mname)
sim.set_all_data_external(check_data=True)
sim.write_simulation()


In [ ]:
os.listdir(tmp_d)

OK!  So we see now that `tmp_d` contains all the MF6 input files and both the array and list quantities are in external files.  Let's make sure the model will run (always important!)

In [ ]:
pyemu.os_utils.run("mf6",cwd=tmp_d)

Now we are ready to start setting up for PEST(++) with `PstFrom`.  First, let's copy dependecies into the `tmp_d` directory so that they will be carried along|

In [ ]:
shutil.copytree(os.path.join("..","..","dependencies","flopy"),os.path.join(tmp_d,"flopy"))
shutil.copytree(os.path.join("..","..","dependencies","pyemu"),os.path.join(tmp_d,"pyemu"))
shutil.copy2("helpers.py",os.path.join(tmp_d,"helpers.py"))
shutil.copy2(os.path.join("..","..","bin","mf6.exe"),os.path.join(tmp_d,"mf6.exe"))
shutil.copy2(os.path.join("..","..","bin","pestpp-ies.exe"),os.path.join(tmp_d,"pestpp-ies.exe"))


Now let's create our `PstFrom` instance.  It will copy the `tmp_d` to `t_d` directory and then setup the pest interface files in `t_d`, leaving `tmp_d` untouched (nice!).  

In [ ]:
t_d = os.path.join("..","..","models","template")
pf = pyemu.utils.PstFrom(tmp_d,t_d,remove_existing=True,spatial_reference=m.modelgrid,zero_based=False,echo=False)

Now we should have a complete set of model files in `template`:

In [ ]:
os.listdir(t_d)

We see that all the same files from `tmp_d` and now in `t_d`, with the addition of the `flopy` and `pyemu` directories.  Now we are going to create a geostatistical structure that will be used to give us coherent correlation in spatially distributed parameter types (to keep the geologist happy!).  In the absense of direct variogram information, we are going to generate a variogram range that is a function of the cell-spacing of the model...

In [ ]:
a = min(m.dis.delr.data.min(),m.dis.delc.data.min()) * min(m.dis.ncol.data,m.dis.nrow.data) * 0.25
print(a)
# creat an exponential variogram
v = pyemu.geostats.ExpVario(contribution=1.0,a=a)
# create a geostruct with the variogram
gs = pyemu.geostats.GeoStruct(variograms=v,transform="log")

So that is our correlation length.Let's take a look at the geostatistical structure in graphical form:

In [ ]:
_ = gs.plot()

Sweet!  Now lets add some parameters.  We will focus on horizontal hydraulic conductivity because, well, we are groundwater modellers and we are crazy about HK!


In [ ]:
tag = "npf_k"
files = [f for f in os.listdir(t_d) if tag in f.lower() and f.endswith(".txt")]
print(files)

So those are the array files for MF6 that are for HK.  Let's do something fancy:  let's setup multiple spatial scales of parameters for HK.  The coarse scale will be a `constant` single value for each array.  The medium scale will pilot points and the finest scale will use parameters as the `grid` scale - each model cell!  Each scale of parameters will work with the others as multipliers with the existing HK arrays - this all happens at runtime.  

In [ ]:
for f in files:
    pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=f.split('.')[1]+"_gr",pargp=f.split('.')[1]+"_gr",
                     lower_bound=0.2,upper_bound=5.0)
    pf.add_parameters(f,par_type="constant",geostruct=gs,par_name_base=f.split('.')[1]+"_cn",
                      pargp=f.split('.')[1]+"_cn",
                     lower_bound=0.2,upper_bound=5.0)
    pf.add_parameters(f,par_type="pilotpoints",geostruct=gs,par_name_base=f.split('.')[1]+"_pp",
                      pargp=f.split('.')[1]+"_pp",
                     lower_bound=0.2,upper_bound=5.0,pp_space=4)

Boom! that was easy...now lets do the same for recharge (because we can!):

In [ ]:
tag = "rcha_recharge"
files = [f for f in os.listdir(t_d) if tag in f.lower() and f.endswith(".txt")]
print(files)
for f in files:
    pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=f.split('.')[1]+"_gr",
                      pargp=f.split('.')[1]+"_gr",
                     lower_bound = 0.8,upper_bound=1.2)
    pf.add_parameters(f,par_type="constant",geostruct=gs,par_name_base=f.split('.')[1]+"_cn",
                      pargp=f.split('.')[1]+"_cn",
                     lower_bound=0.8,upper_bound=1.2)
    pf.add_parameters(f,par_type="pilotpoints",geostruct=gs,par_name_base=f.split('.')[1]+"_pp",
                      pargp=f.split('.')[1]+"_pp",
                     lower_bound=0.8,upper_bound=1.2,pp_space=4)
    

And dont forget transport properties:

In [ ]:

tag = "mst_porosity"
files = [f for f in os.listdir(t_d) if tag in f.lower() and f.endswith(".txt")]
print(files)
for f in files:
    pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=f.split('.')[1]+"_gr",
                      pargp=f.split('.')[1]+"_gr",
                     lower_bound = 0.8,upper_bound=1.2,ult_ubound=0.2,ult_lbound=0.01)
    pf.add_parameters(f,par_type="constant",geostruct=gs,par_name_base=f.split('.')[1]+"_cn",
                      pargp=f.split('.')[1]+"_cn",
                     lower_bound=0.8,upper_bound=1.2,ult_ubound=0.2,ult_lbound=0.01)
    pf.add_parameters(f,par_type="pilotpoints",geostruct=gs,par_name_base=f.split('.')[1]+"_pp",
                      pargp=f.split('.')[1]+"_pp",
                     lower_bound=0.8,upper_bound=1.2,pp_space=4,ult_ubound=0.2,ult_lbound=0.01)

Now this will make some people uncomfortable but how well do we really ever know historic water use flux rates in space and in time? hmmm, not really! So lets add parameters to represent that uncertainty in the model inputs: 

In [ ]:
tag = "wel_stress_period_data"
files = [f for f in os.listdir(t_d) if tag in f.lower() and f.endswith(".txt")]
print(files)
for f in files:
    kper = int(f.split('.')[1].split('_')[-1])
    name = "welflux_{0:04d}".format(kper)
    pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=name,pargp=name,
                     index_cols=[0,1,2],use_cols=[3],lower_bound=0.5,upper_bound=1.5)

What about those ghb stages along the boundaries of the model?  Maybe we should consider their uncertainty also?  Since these boundaries are likely to be very influential, we want to include a robust representation of their uncertainty - both stage and conductance and at multiple scales.  

In [ ]:
tag = "ghb_stress_period_data"
files = [f for f in os.listdir(t_d) if tag in f.lower() and f.endswith(".txt")]
print(files)
for f in files:
    kper = int(f.split('.')[1].split('_')[-1])
    
    # constant and grid scale multiplier conductance parameters
    name = "ghbcond_{0:04d}".format(kper)
    pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=name+"_gr",pargp=name+"_gr",
                     index_cols=[0,1,2],use_cols=[4],lower_bound=0.1,upper_bound=10.0)
    pf.add_parameters(f,par_type="constant",geostruct=gs,par_name_base=name+"_cn",pargp=name+"_cn",
                     index_cols=[0,1,2],use_cols=[4],lower_bound=0.1,upper_bound=10.0)
    
    # constant and grid scale additive stage parameters
    name = "ghbstage_{0:04d}".format(kper)
    pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=name+"_gr",pargp=name+"_gr",
                     index_cols=[0,1,2],use_cols=[3],par_style="a",lower_bound=-2.0,upper_bound=2.0,
                     transform="none")
    pf.add_parameters(f,par_type="constant",geostruct=gs,par_name_base=name+"_cn",pargp=name+"_cn",
                     index_cols=[0,1,2],use_cols=[3],lower_bound=-2.0,upper_bound=2.0,par_style="a",
                     transform="none")
    

and who could forget SFR conductance!

In [ ]:
tag = "sfr_packagedata"
files = [f for f in os.listdir(t_d) if tag in f.lower() and f.endswith(".txt")]
assert len(files) == 1
print(files)
f = files[0]
# constant and grid scale multiplier conductance parameters
name = "sfrcond"
pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=name+"_gr",pargp=name+"_gr",
                 index_cols=[0,2,3],use_cols=[9],lower_bound=0.1,upper_bound=10.0)
pf.add_parameters(f,par_type="constant",geostruct=gs,par_name_base=name+"_cn",pargp=name+"_cn",
                 index_cols=[0,2,3],use_cols=[9],lower_bound=0.1,upper_bound=10.0)



And let's also consider source loading concentration

In [ ]:
tag = "cnc_stress_period_data"
files = [f for f in os.listdir(t_d) if tag in f.lower() and f.endswith(".txt")]
assert len(files) == 1
print(files)
f = files[0]
name = "cnc"
pf.add_parameters(f,par_type="grid",geostruct=gs,par_name_base=name+"_gr",pargp=name+"_gr",
                 index_cols=[0,1,2],use_cols=[3],lower_bound=0.1,upper_bound=10.0)
pf.add_parameters(f,par_type="constant",geostruct=gs,par_name_base=name+"_cn",pargp=name+"_cn",
                 index_cols=[0,1,2],use_cols=[3],lower_bound=0.1,upper_bound=10.0)

Sweet!  thats heaps of parameters - exactly what we wanted.  Now lets setup some observations in the pest control file.  To start, we need to run a simple post processor that will extract the simulated water levels from the binary headsave file and save to an ASCII file, and also process the list budget file into csv files.  Let's see what is in this helpers script:

In [ ]:
[l.strip() for l in open("helpers.py",'r').readlines()]

Just some basic python hackery there! The top bit extracts the simulated groundwater levels from the binary headsave file and saves to an ASCII array (why can't MF6 save ASCII???) and the lower part parses the list file for volume budget information. 

Now lets run this function in the `t_d` directory to generate some output files that we can use for setting up observations:

In [ ]:
pyemu.os_utils.run("python helpers.py",cwd=t_d)

first lets add some sfr observations

In [ ]:
df = pf.add_observations(mname+".sfr.obs.output.csv",index_cols=0,prefix="sw")

add the MF6 head obs output file

In [ ]:
df = pf.add_observations(mname+".head.obs",index_cols=[0],prefix="head",ofile_sep=',')

and MF6 concentration obs output file

In [ ]:
df = pf.add_observations("conc_obs.csv",index_cols=[0],prefix="concen",ofile_sep=',')

lets add a pest observation for every active model cell.  Why?  because we can! and because in an ensemble-based workflow, the cost of getting just one new simulated output is high, so its easier to just carry all the outputs you can think of - its just storage since these outputs arent used for any calculations

In [ ]:
hds_files = [f for f in os.listdir(t_d) if f.endswith("hds.dat")]
assert len(hds_files) == m.dis.nlay.data * sim.tdis.nper.data,len(hds_files)
for hds_file in hds_files:
    pf.add_observations(hds_file,obsgp=hds_file.split('.')[1],prefix=hds_file.split('.')[1])

And the same for simulated concentrations....

In [ ]:
ucn_files = [f for f in os.listdir(t_d) if f.endswith("ucn.dat")]
assert len(ucn_files) == m.dis.nlay.data * sim.tdis.nper.data,len(ucn_file)
for ucn_file in ucn_files:
    pf.add_observations(ucn_file,obsgp=ucn_file.split('.')[1],prefix=ucn_file.split('.')[1])

And also observations for the cumulative and incremental list file budget - these are great diagnostic quantities to keep track of!

In [ ]:
df = pd.read_csv(os.path.join(t_d,"inc.csv"),index_col=0)
pf.add_observations("inc.csv",index_cols=["time"],use_cols=df.columns.tolist(),obsgp="inc")
df = pd.read_csv(os.path.join(t_d,"cum.csv"),index_col=0)
pf.add_observations("cum.csv",index_cols=["time"],use_cols=df.columns.tolist(),obsgp="cum")

In [ ]:
df = pd.read_csv(os.path.join(t_d,"tcum.csv"),index_col=0)
pf.add_observations("tcum.csv",index_cols=["time"],use_cols=df.columns.tolist(),obsgp="tcum")
df = pd.read_csv(os.path.join(t_d,"tinc.csv"),index_col=0)
pf.add_observations("tinc.csv",index_cols=["time"],use_cols=df.columns.tolist(),obsgp="tinc")

Always a good idea to remove intermediate processing files to help prevent them getting used erroneously

In [ ]:
pf.tmp_files.append(mname+".hds")
pf.tmp_files.append("gwt-sgn.ucn")
pf.tmp_files.append(mname+".list")
pf.tmp_files.append("gwt-sgn.list")

Now we need to tell `PstFrom` that we want that `helpers.py` script to run after MF6, every time that MF6 runs. We will use a method that reads python source files and extracts functions (wat?!):

In [ ]:
pf.add_py_function("helpers.py","postproc()",is_pre_cmd=False)

Now we just need to set the model run command

In [ ]:
pf.mod_sys_cmds.append("mf6")

Magic time!:

In [ ]:
pf.build_pst()

In [ ]:
[f for f in os.listdir(t_d) if f[-3:] in ["pst","tpl","ins",".py"]]

Everything we need to run PEST(++) is now in `t_d`.   Let's checkout `temp.pst`: 

In [ ]:

pf.pst.npar,pf.pst.nobs

We just built a very-high dimensional PEST interface - snap!

Now, lets generate a prior parameter ensemble:

In [ ]:
pe = pf.draw(num_reals=100,use_specsim=True)
pe.enforce()
pe.to_binary(os.path.join(t_d,"prior.jcb"))
pf.pst.pestpp_options["ies_parameter_ensemble"] = "prior.jcb"
pf.pst.write(os.path.join(t_d,"sgn.pst"),version=2)

Boom!  We now have a geostatistically correlated prior parameter ensemble.

We can use all the standard `pandas` action the `ParameterEnsemble` instance:

In [ ]:
ax = pe.iloc[:,0].hist()
_ = ax.set_title(pe.columns[0])

In the next notebook, we will make some test runs of `pestpp-ies` and also explore the multiplier parameter process a little bit more...